In [2]:
import os
import re
import warnings
import random
from collections import defaultdict
from typing import Dict, List, Tuple

import torch
import torch.nn.functional as F
import numpy as np
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or
/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [3]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

## Задание

1) Реализовать методы `greedy_sampling` и `generate` (1 балл)
2) Реализовать метод `random_sampling` и поддержать его в `generate` (1 балл)
3) Реализовать метод `_beam_search_generate` и поддержать его в `generate` (2 балла)
4) Реализовать методы `apply_top_p`, `apply_top_k`, `apply_temperature` и поддержать их в `generate` (1 балл)  
Все методы необходимо реализовать через векторные операции в torch/numpy везде где это возможно

In [4]:
class Model:
    def __init__(self, model_name: str = "gpt2"):
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.vocab_size = self.tokenizer.vocab_size
        self.model.eval()

    def greedy_sampling(self, logits: torch.Tensor) -> int:
        return torch.argmax(logits, dim=-1).item()

    def random_sampling(self, logits: torch.Tensor) -> int:
        probs = F.softmax(logits, dim=-1)
        return torch.multinomial(probs, num_samples=1).item()

    def _beam_search_generate(
        self,
        prompt: str,
        max_length: int,
        num_beams: int
    ) -> str:
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt')
        beams = [(input_ids, 0.0)]

        with torch.no_grad():
            for _ in range(max_length):
                all_candidates = []

                for seq, score in beams:
                    if seq[0, -1].item() == self.tokenizer.eos_token_id:
                        all_candidates.append((seq, score))
                        continue

                    outputs = self.model(seq)
                    logits = outputs.logits[0, -1, :]

                    log_probs = F.log_softmax(logits, dim=-1)
                    top_log_probs, top_indices = torch.topk(log_probs, num_beams)

                    for log_prob, token_id in zip(top_log_probs, top_indices):
                        new_seq = torch.cat([seq, token_id.unsqueeze(0).unsqueeze(0)], dim=1)
                        new_score = score + log_prob.item()
                        all_candidates.append((new_seq, new_score))

                beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:num_beams]
                if all(beam[0][0, -1].item() == self.tokenizer.eos_token_id for beam in beams):
                    break

        best_seq = beams[0][0]
        return self.tokenizer.decode(best_seq[0], skip_special_tokens=True)


    def apply_temperature(self, logits: torch.Tensor, temperature: float = 1.0) -> torch.Tensor:
        return logits / temperature

    def _apply_top_k(self, logits: torch.Tensor, top_k: int = 0) -> torch.Tensor:
        if top_k <= 0 or top_k >= self.vocab_size:
            return logits

        # Находим k-й наибольший элемент
        top_k_values, _ = torch.topk(logits, top_k)
        threshold = top_k_values[-1]

        # Маскируем все значения меньше порога
        logits_filtered = logits.clone()
        logits_filtered[logits < threshold] = float('-inf')

        return logits_filtered

    def _apply_top_p(self, logits: torch.Tensor, top_p: float = 1.0) -> torch.Tensor:
         if top_p >= 1.0:
            return logits

         sorted_logits, sorted_indices = torch.sort(logits, descending=True)

         cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

         sorted_indices_to_remove = cumulative_probs > top_p

         sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1].clone()
         sorted_indices_to_remove[0] = False

         logits_filtered = logits.clone()
         indices_to_remove = sorted_indices[sorted_indices_to_remove]
         logits_filtered[indices_to_remove] = float('-inf')

         return logits_filtered

    def generate(
        self,
        prompt: str,
        max_length: int = 50,
        strategy: str = "greedy",
        temperature: float = 1.0,
        top_k: int = 0,
        top_p: float = 1.0,
        num_beams: int = 3
    ) -> str:

        if strategy == "beam_search":
            return self._beam_search_generate(prompt, max_length, num_beams)

        input_ids = self.tokenizer.encode(prompt, return_tensors='pt')

        with torch.no_grad():
            for _ in range(max_length):
                # получаем логиты от модели
                outputs = self.model(input_ids)
                logits = outputs.logits[0, -1, :]  # (vocab_size,)

                # применяем temperature scaling
                logits = self.apply_temperature(logits, temperature)

                # применяем top-k фильтрацию
                if top_k > 0:
                    logits = self._apply_top_k(logits, top_k)

                # применяем top-p фильтрацию
                if top_p < 1.0:
                    logits = self._apply_top_p(logits, top_p)

                # выбираем следующий токен в зависимости от стратегии
                if strategy == "greedy":
                    next_token_id = self.greedy_sampling(logits)
                elif strategy == "random":
                    next_token_id = self.random_sampling(logits)
                else:
                    raise ValueError(f"Unknown strategy: {strategy}")

                # добавляем токен к последовательности
                input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]])], dim=1)

                # останавливаемся при встрече EOS токена
                if next_token_id == self.tokenizer.eos_token_id:
                    break

        # декодируем и возвращаем результат
        return self.tokenizer.decode(input_ids[0], skip_special_tokens=True)



In [ ]:
# Продемонстрируйте результат работы `generate` при различных параметрах

In [6]:
model = Model("gpt2")
prompt = "once upon a time"

print(" Greedy Sampling ------------------------")
print(model.generate(prompt, max_length=30, strategy="greedy"))

print("\n Random Sampling ------------------------")
print(model.generate(prompt, max_length=30, strategy="random", temperature=0.8))

print("\n Random with Top-K---------------------------")
print(model.generate(prompt, max_length=30, strategy="random", top_k=50))

print("\n Random with Top-P -----------------------")
print(model.generate(prompt, max_length=30, strategy="random", top_p=0.9))

print("\n Beam Search ----------------------")
print(model.generate(prompt, max_length=30, strategy="beam_search", num_beams=5))

 Greedy Sampling ------------------------
once upon a time, and I'm not sure if it's a coincidence or not.

I'm not sure if it's a coincidence or not. I'm

 Random Sampling ------------------------
once upon a time?

I didn't have an ounce of doubt, I knew it was not a good idea.

But I am strong, I look

 Random with Top-K---------------------------
once upon a time it was all a lie. It would be such a good bet to live it up."

After a couple of weeks of therapy, they had

 Random with Top-P -----------------------
once upon a time, it was when night fell upon my place in myself to defend myself against them. I went off to find my wife and children, and among them

 Beam Search ----------------------
once upon a time.

"I'm not going to lie to you, I'm not going to lie to you, I'm not going to lie to you


In [8]:
prompt = "what is in epstein files"

print(" Greedy Sampling ------------------------")
print(model.generate(prompt, max_length=30, strategy="greedy"))

print("\n Random Sampling ------------------------")
print(model.generate(prompt, max_length=30, strategy="random", temperature=0.8))

print("\n Random with Top-K---------------------------")
print(model.generate(prompt, max_length=30, strategy="random", top_k=50))

print("\n Random with Top-P -----------------------")
print(model.generate(prompt, max_length=30, strategy="random", top_p=0.9))

print("\n Beam Search ----------------------")
print(model.generate(prompt, max_length=30, strategy="beam_search", num_beams=5))

 Greedy Sampling ------------------------
what is in epstein files?

The epstein file is a collection of files that are stored in the epstein directory. The files are stored in the epstein directory

 Random Sampling ------------------------
what is in epstein files that are not in the archive?)

Added NULL to WebObject definitions to avoid naming conflicts with others

Added methods for setting height to 100

 Random with Top-K---------------------------
what is in epstein files) :

- I just found out the new format that it comes in seems less buggy. I had to fix most tests.

-

 Random with Top-P -----------------------
what is in epstein files and what's in sync with HLSF

Advantages

These compression ratios give a good separation between the image and buffer. I

 Beam Search ----------------------
what is in epstein files?

The epstein file is a collection of epstein files. The epstein file is a collection of epstein files. The epstein
